In [1]:
!gdown --id 1OurDQUtbWQacvT32HMqFL7vIUrSMllOp

Downloading...
From: https://drive.google.com/uc?id=1OurDQUtbWQacvT32HMqFL7vIUrSMllOp
To: /content/preprocessed_data.csv
100% 300k/300k [00:00<00:00, 9.69MB/s]


In [8]:
!pip install kaggle


In [9]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d yekenot/fasttext-crawl-300d-2m


mkdir: cannot create directory ‘/root/.kaggle’: File exists
 99% 1.43G/1.44G [00:06<00:00, 234MB/s]
100% 1.44G/1.44G [00:06<00:00, 223MB/s]


In [10]:
!7z e fasttext-crawl-300d-2m.zip -o/content -r



7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1545551987 bytes (1474 MiB)

Extracting archive: fasttext-crawl-300d-2m.zip
--
Path = fasttext-crawl-300d-2m.zip
Type = zip
Physical Size = 1545551987

  0%      0% - crawl-300d-2M.vec                          1% - crawl-300d-2M.vec                          2% - crawl-300d-2M.vec                          3% - crawl-300d-2M.vec                          4% - crawl-300d-2M.vec                          5% - crawl-300d-2M.vec                        

In [11]:
#Importing necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [12]:
# Reading glove vectors in python: https://stackoverflow.com/a/38230349/4084039
def fasttextModel(gloveFile):
    print ("Loading Fasttext Model")
    f = open(gloveFile,'r', encoding="utf8")
    model = {}#for storing word and the corresponding embedding vector for that word
    for line in f:
        splitLine = line.split()#splitting the line and storing it in a list
        word = splitLine[0]#getting the first element and storing it in word
        embedding = np.array([float(val) for val in splitLine[1:]])#obtaining corresponding vector for that word
        model[word] = embedding#storing word as key and embedding vector for that word as value
    print ("Done.",len(model)," words loaded!")
    return model
model = fasttextModel('/content/crawl-300d-2M.vec')

Loading Fasttext Model
Done. 2000000  words loaded!


In [13]:
df=pd.read_csv('preprocessed_data.csv')#creating DataFrame using preprocessed_data.csv

In [14]:
df.head(4)

,Unnamed: 0,source,target
0,0,"U wan me to ""chop"" seat 4 u nt?\n",Do you want me to reserve seat for you or not?\n
1,1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,3,I'm thai. what do u do?\n,I'm Thai. What do you do?\n


In [15]:
def preprocess(x):#removing last character
  x=x[:-1]
  return x

In [16]:
df['source']=df['source'].apply(preprocess)#preprocessing source data
df['target']=df['target'].apply(preprocess)#preprocessing target data

In [17]:
df=df[['source','target']]
df.head()

,source,target
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [18]:
df.shape

(2000, 2)

In [19]:
def length(text):#for calculating the length of the sentence
    return len(str(text))

In [20]:
df=df[df['source'].apply(length)<170]#removing the datapoints where the source sentence length is greater than or equal to 170
df=df[df['target'].apply(length)<200]#removing the datapoints where the source sentence length is greater than or equal to 170

In [21]:
df.shape

(1990, 2)

In [22]:
df['target_in'] = '<start> ' + df['target'].astype(str)
df['target_out'] = df['target'].astype(str) + ' <end>'
# only for the first sentance add a toke <end> so that we will have <end> in tokenizer
df.head()

,source,target,target_in,target_out
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?,<start> Do you want me to reserve seat for you...,Do you want me to reserve seat for you or not?...
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...,<start> Yeap. You reaching? We ordered some Du...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...,<start> They become more expensive already. Mi...,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?,<start> I'm Thai. What do you do?,I'm Thai. What do you do? <end>
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...,<start> Hi! How did your week go? Haven't hear...,Hi! How did your week go? Haven't heard from y...


In [23]:
df=df.drop('target',axis=1)#removing the target column


In [24]:
df.head(4)

,source,target_in,target_out
0,"U wan me to ""chop"" seat 4 u nt?",<start> Do you want me to reserve seat for you...,Do you want me to reserve seat for you or not?...
1,Yup. U reaching. We order some durian pastry a...,<start> Yeap. You reaching? We ordered some Du...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,<start> They become more expensive already. Mi...,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,<start> I'm Thai. What do you do?,I'm Thai. What do you do? <end>


In [25]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(df, test_size=0.01)#splitting the data in ratio 99:1

In [26]:
print(train.shape, validation.shape)
# for one sentence we will be adding <end> token so that the tokanizer learns the word <end>
# with this we can use only one tokenizer for both encoder output and decoder output
train.iloc[0]['target_in']= str(train.iloc[0]['target_in'])+' <end>'
train.iloc[0]['target_out']= str(train.iloc[0]['target_out'])+' <end>'

(1970, 3) (20, 3)


In [27]:
tknizer_source = Tokenizer()#creating tokenziation
tknizer_source.fit_on_texts(train['source'].values)#fitting on source data
tknizer_target = Tokenizer(filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n')#creating tokenziation
tknizer_target.fit_on_texts(train['target_in'].values)#fitting on target data

In [28]:
vocab_size_target=len(tknizer_target.word_index.keys())#target vocab size
print(vocab_size_target)
vocab_size_source=len(tknizer_source.word_index.keys())#source vocab size
print(vocab_size_source)

3040
3711


In [29]:
tknizer_target.word_index['<start>'], tknizer_target.word_index['<end>']

(1, 1440)

In [30]:
encoder_embedding_matrix = np.zeros((vocab_size_source+1, 300))
for word, i in tknizer_source.word_index.items():
    embedding_vector = model.get(word)
    if embedding_vector is not None:
        encoder_embedding_matrix[i] = embedding_vector

In [31]:
decoder_embedding_matrix = np.zeros((vocab_size_target+1, 300))
for word, i in tknizer_target.word_index.items():
    embedding_vector = model.get(word)
    if embedding_vector is not None:
        decoder_embedding_matrix[i] = embedding_vector

In [70]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns encoder-outputs,encoder_final_state_h,encoder_final_state_c
    '''

    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):

        #Initialize Embedding layer
        #Intialize Encoder LSTM layer
        super().__init__()
        self.vocab_size = inp_vocab_size
        self.embedding_size = embedding_size
        self.input_length = input_length
        self.lstm_size= lstm_size
        self.lstm_output=0
        self.lstm_state_h=0
        self.lstm_state_c=0
        self.embedding = tf.keras.layers.Embedding(input_dim=self.vocab_size, output_dim=self.embedding_size, input_length=self.input_length,
                           mask_zero=True,name="embedding_layer_encoder",weights=[encoder_embedding_matrix],trainable=False)
        self.lstm = tf.keras.layers.LSTM(self.lstm_size, return_state=True, return_sequences=True, name="Encoder_LSTM")

    def call(self,input_sequence,states):
        '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- encoder_output, last time step's hidden and cell state
        '''
        
        input_embedd                           = self.embedding(input_sequence)
        lstm_state_h,lstm_state_c = states[0],states[1]
        self.lstm_output,lstm_state_h,lstm_state_c=self.lstm(input_embedd)
        return self.lstm_output,lstm_state_h,lstm_state_c

      

    
    def initialize_states(self,batch_size):
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      return [np.zeros((batch_size,self.lstm_size)),np.zeros((batch_size,self.lstm_size))]


In [71]:
class Decoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,out_vocab_size,embedding_size,lstm_size,input_length):

        #Initialize Embedding layer
        #Intialize Decoder LSTM layer
        super().__init__()
        self.out_vocab_size = out_vocab_size
        self.embedding_size = embedding_size
        self.lstm_size = lstm_size
        self.input_length = input_length
        # we are using embedding_matrix and not training the embedding layer
        self.embedding = tf.keras.layers.Embedding(input_dim=self.out_vocab_size, output_dim=self.embedding_size, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_decoder",weights=[decoder_embedding_matrix],trainable=False)
        self.gru = tf.keras.layers.GRU(self.lstm_size, return_sequences=True, return_state=True, name="Encoder_LSTM")
    
    def call(self,input_sequence,initial_states):
        '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to decoder_lstm
        
          returns -- decoder_output,decoder_final_state_h,decoder_final_state_c
        '''
        target_embedd = self.embedding(input_sequence)
        decoder_output,decoder_final_state_h = self.gru(target_embedd, initial_state=[initial_states[0]])
        return decoder_output,decoder_final_state_h

      


In [80]:
class Encoder_decoder(tf.keras.Model):
    
    def __init__(self,encoder_inputs_length,decoder_inputs_length, output_vocab_size,batch_size):
        
        #Create encoder object
        #Create decoder object
        #Intialize Dense layer(out_vocab_size) with activation='softmax'
        super().__init__() # https://stackoverflow.com/a/27134600/4084039
        self.batch_size=batch_size
        self.encoder = Encoder(vocab_size_source+1,300,256,encoder_inputs_length)
        self.decoder = Decoder(vocab_size_target+1,300,256,decoder_inputs_length)
        self.dense   = tf.keras.layers.Dense(output_vocab_size, activation='softmax')
    
    
    def call(self,data):
        '''
        A. Pass the input sequence to Encoder layer -- Return encoder_output,encoder_final_state_h,encoder_final_state_c
        B. Pass the target sequence to Decoder layer with intial states as encoder_final_state_h,encoder_final_state_C
        C. Pass the decoder_outputs into Dense layer 
        
        Return decoder_outputs
        '''
        input,output = data[0], data[1]
        initial_state=self.encoder.initialize_states(self.batch_size)
        encoder_output, encoder_h, encoder_c = self.encoder(input,initial_state)
        decoder_output,decoder_final_state_h= self.decoder(output,[encoder_h])
        output                               = self.dense(decoder_output)
        return output
        

In [81]:
class Dataset:
    def __init__(self, df, tknizer_source, tknizer_target, source_len,target_len):
        self.encoder_inps = df['source'].values
        self.decoder_inps = df['target_in'].values
        self.decoder_outs = df['target_out'].values
        self.tknizer_target = tknizer_target
        self.tknizer_source = tknizer_source
        self.source_len = source_len
        self.target_len = target_len


    def __getitem__(self, i):
        self.encoder_seq = self.tknizer_source.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tknizer_target.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_target.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.source_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.target_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.target_len, dtype='int32', padding='post')
        self.one_hot_encoded = np.zeros((len(self.decoder_out_seq),self.target_len,vocab_size_target),dtype='float32')

        for i,sentence in enumerate(self.decoder_out_seq):
          for j,word in enumerate(sentence):
            self.one_hot_encoded[i,j,word]=1
        



        return self.encoder_seq, self.decoder_inp_seq, self.one_hot_encoded

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)

    
class Dataloder(tf.keras.utils.Sequence):    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        return tuple([[batch[0],batch[1]],batch[2]])

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [82]:
train_dataset = Dataset(train, tknizer_source, tknizer_target,39,43)
test_dataset  = Dataset(validation, tknizer_source, tknizer_target,39,43)

train_dataloader = Dataloder(train_dataset, batch_size=512)
test_dataloader = Dataloder(test_dataset, batch_size=20)


print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(512, 39) (512, 43) (512, 43, 3040)


In [83]:
tf.config.experimental_run_functions_eagerly(True)


In [84]:
tf.config.run_functions_eagerly(True)


In [85]:
def changeLearningRate(epoch,lr):
    if epoch % 3 == 0:
      return lr*(0.5)
    return lr

In [86]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,LearningRateScheduler
import datetime

early_stop = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
check_point = ModelCheckpoint('best_model_1.h5', monitor='val_loss',  verbose=1, save_best_only=True, mode='min',save_format='tf')
lrschedule = LearningRateScheduler(changeLearningRate, verbose=1)

In [87]:
#Create an object of encoder_decoder Model class, 
# Compile the model and fit the model
model  = Encoder_decoder(encoder_inputs_length=39,decoder_inputs_length=43,output_vocab_size=vocab_size_target,batch_size=512)
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
train_steps=train.shape[0]//512
valid_steps=validation.shape[0]//20
model.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=100, validation_data=test_dataloader, validation_steps=valid_steps,callbacks=[early_stop,lrschedule])
model.summary()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/100

Epoch 00001: LearningRateScheduler reducing learning rate to 0.004999999888241291.
3/3 [==============================] - 1s 422ms/step - loss: 2.7390 - accuracy: 0.0507 - val_loss: 1.6437 - val_accuracy: 0.0946
Epoch 2/100

Epoch 00002: LearningRateScheduler reducing learning rate to 0.004999999888241291.
3/3 [==============================] - 1s 399ms/step - loss: 2.1927 - accuracy: 0.0677 - val_loss: 1.5412 - val_accuracy: 0.0946
Epoch 3/100

Epoch 00003: LearningRateScheduler reducing learning rate to 0.004999999888241291.
3/3 [==============================] - 1s 406ms/step - loss: 2.1509 - accuracy: 0.0669 - val_loss: 1.5111 - val_accuracy: 0.0901
Epoch 4/100

Epoch 00004: LearningRateScheduler reducing learning rate to 0.0024999999441206455.
3/3 [==============================] - 1s 412ms/step - loss: 2.0908 - accuracy: 0.0870 - val_loss: 1.5020 - val_accuracy: 0.1081
Epoch 5/100

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0024999999441206455.
3/

In [88]:
batch_size=512
units=256

In [92]:
def predict(input_sentence):

  '''
  A. Given input sentence, convert the sentence into integers using tokenizer used earlier
  B. Pass the input_sequence to encoder. we get encoder_outputs, last time step hidden and cell state
  C. Initialize index of <start> as input to decoder. and encoder final states as input_states to decoder
  D. till we reach max_length of decoder or till the model predicted word <end>:
         predicted_out,state_h,state_c=model.layers[1](dec_input,states)
         pass the predicted_out to the dense layer
         update the states=[state_h,state_c]
         And get the index of the wordcc with maximum probability of the dense layer output, using the tokenizer(word index) get the word and then store it in a string.
         Update the input_to_decoder with current predictions
  F. Return the predicted sentence
  '''
  initial_state_enc=[np.zeros((batch_size,units)),np.zeros((batch_size,units))]
  inp_seq = tknizer_source.texts_to_sequences([input_sentence])
  inp_seq = pad_sequences(inp_seq,padding='post',maxlen=39)

  en_outputs,state_h , state_c = model.layers[0](tf.constant(inp_seq),initial_state_enc)
  cur_vec = tf.constant([[tknizer_target.word_index['<start>']]])
  pred = []
  #Here 43 is the max_length of the sequence
  for i in range(43):
    infe_output, state_h = model.layers[1](cur_vec,[state_h])
    infe_output = model.layers[2](infe_output)
    cur_vec = np.reshape(np.argmax(infe_output), (1, 1))
    pred.append(tknizer_target.index_word[cur_vec[0][0]])
    if(pred[-1]=='<end>'):
      break
    translated_sentence = ' '.join(pred)

  return translated_sentence

In [93]:
validation['target_in']

899                         <start> Malays are all woods.
52      <start> Yeap. I will call you in a while? I'm ...
1597              <start> Where? Can I come and find you?
847     <start> No need for me to introduce. Someone a...
189     <start> Ben asks us to wait at the MRT bus sto...
1713                                          <start> Ok.
1818               <start> I'll be late. I will call you.
591                 <start> 2:30 then. Where will you be?
1450    <start> Haha. Hey, MERINA is my name. OK, fema...
175          <start> Joey: Hi! Does anybody want to chat?
1136                                     <start> No need.
786                     <start> Yes. Sure. Evening right?
1966                    <start> Oops. Sorry. I am coming.
1144    <start> Sigh, what's new man? So this is her n...
35      <start> I am working in NTUC Income, selling i...
1821    <start> Hey, are you in the LT already? I'm on...
1139    <start> You worse than me. Eat more of everyth...
1103    <start

In [94]:
for i in validation['source']:
  predicted=predict(i)
  print("The predicted output is: ",predicted)


The predicted output is:  i don't know you you you you
The predicted output is:  hey i am you you you you you you you you
The predicted output is:  i don't know you you you you you you you
The predicted output is:  i don't know you you you you you you you
The predicted output is:  hey i am you you you you you you you you
The predicted output is:  you you you you
The predicted output is:  i don't know you you you you you you you
The predicted output is:  i don't know you you you you you you you
The predicted output is:  i don't know you you you you you you you
The predicted output is:  i don't know you you you you you you you
The predicted output is:  i don't know you you you you you you
The predicted output is:  i don't know you you you you you you
The predicted output is:  i don't know you you you you you you you
The predicted output is:  i don't know you you you you you you you
The predicted output is:  hey i am you you you you you you you you
The predicted output is:  i don't know y

In [95]:
# Predict on 1000 random sentences on test data and calculate the average BLEU score of these sentences.
import nltk.translate.bleu_score as bleu
bleu_scores_lst=[]
for i in validation[:]['source']:
  reference = [i.split(),] # the original
  predicted=predict(i)
  translation = predicted.split()
  values=bleu.sentence_bleu(reference, translation)
  bleu_scores_lst.append(values)

# https://www.nltk.org/_modules/nltk/translate/bleu_score.html

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [96]:
average_bleu_scores=sum(bleu_scores_lst)/len(bleu_scores_lst)
print("Average BLEU score of these 20 test data sentences is: ",average_bleu_scores)

Average BLEU score of these 20 test data sentences is:  0.059428793291298146


In [97]:
bleu_scores_lst


[0,
 0.4578141331660858,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0.16842040746952797,
 0,
 0,
 0,
 0.5623413251903491,
 0]

Character_Level:

In [98]:
df=pd.read_csv('preprocessed_data.csv')
df.head()

,Unnamed: 0,source,target
0,0,"U wan me to ""chop"" seat 4 u nt?\n",Do you want me to reserve seat for you or not?\n
1,1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,3,I'm thai. what do u do?\n,I'm Thai. What do you do?\n
4,4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [99]:
def preprocess(x):
  x=x[:-1]
  return x

In [100]:
df['source']=df['source'].apply(preprocess)
df['target']=df['target'].apply(preprocess)

In [101]:
df=df[['source','target']]
df.head()

,source,target
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...


In [102]:
df.shape

(2000, 2)

In [103]:
def length(text):#for calculating the length of the sentence
    return len(str(text))

In [104]:
df=df[df['source'].apply(length)<170]
df=df[df['target'].apply(length)<200]

In [105]:
df.shape


(1990, 2)

In [106]:
df['target_in'] = '\t' + df['target'].astype(str)
df['target_out'] = df['target'].astype(str) + '\n'
# only for the first sentance add a toke <end> so that we will have <end> in tokenizer
df.head()

,source,target,target_in,target_out
0,"U wan me to ""chop"" seat 4 u nt?",Do you want me to reserve seat for you or not?,\tDo you want me to reserve seat for you or not?,Do you want me to reserve seat for you or not?\n
1,Yup. U reaching. We order some durian pastry a...,Yeap. You reaching? We ordered some Durian pas...,\tYeap. You reaching? We ordered some Durian p...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,They become more expensive already. Mine is li...,\tThey become more expensive already. Mine is ...,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,I'm Thai. What do you do?,\tI'm Thai. What do you do?,I'm Thai. What do you do?\n
4,Hi! How did your week go? Haven heard from you...,Hi! How did your week go? Haven't heard from y...,\tHi! How did your week go? Haven't heard from...,Hi! How did your week go? Haven't heard from y...


In [107]:
df=df.drop('target',axis=1)


In [108]:
df.head(4)

,source,target_in,target_out
0,"U wan me to ""chop"" seat 4 u nt?",\tDo you want me to reserve seat for you or not?,Do you want me to reserve seat for you or not?\n
1,Yup. U reaching. We order some durian pastry a...,\tYeap. You reaching? We ordered some Durian p...,Yeap. You reaching? We ordered some Durian pas...
2,They become more ex oredi... Mine is like 25.....,\tThey become more expensive already. Mine is ...,They become more expensive already. Mine is li...
3,I'm thai. what do u do?,\tI'm Thai. What do you do?,I'm Thai. What do you do?\n


In [109]:
from sklearn.model_selection import train_test_split
train, validation = train_test_split(df, test_size=0.01)

In [110]:
print(train.shape, validation.shape)
# for one sentence we will be adding <end> token so that the tokanizer learns the word <end>
# with this we can use only one tokenizer for both encoder output and decoder output
train.iloc[0]['target_in']= str(train.iloc[0]['target_in'])+'\n'
train.iloc[0]['target_out']= str(train.iloc[0]['target_out'])+'\n'

(1970, 3) (20, 3)


In [111]:
tknizer_source = Tokenizer(filters=None,char_level=True,lower=False)
tknizer_source.fit_on_texts(train['source'].values)
tknizer_target = Tokenizer(filters=None,char_level=True,lower=False)
tknizer_target.fit_on_texts(train['target_in'].values)

In [112]:
vocab_size_target=len(tknizer_target.word_index.keys())
print(vocab_size_target)
vocab_size_source=len(tknizer_source.word_index.keys())
print(vocab_size_source)

92
103


In [113]:
tknizer_target.word_index['\t'], tknizer_target.word_index['\n']

(20, 85)

In [122]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns encoder-outputs,encoder_final_state_h,encoder_final_state_c
    '''

    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length):

        #Initialize Embedding layer
        #Intialize Encoder LSTM layer
        super().__init__()
        self.vocab_size = inp_vocab_size
        self.embedding_size = embedding_size
        self.input_length = input_length
        self.lstm_size= lstm_size
        self.lstm_output=0
        self.lstm_state_h=0
        self.lstm_state_c=0
        self.embedding = tf.keras.layers.Embedding(input_dim=self.vocab_size, output_dim=self.embedding_size, input_length=self.input_length,
                           mask_zero=True,name="embedding_layer_encoder")
        self.lstm = tf.keras.layers.LSTM(self.lstm_size, return_state=True, return_sequences=True, name="Encoder_LSTM")

    def call(self,input_sequence,states):
        '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- encoder_output, last time step's hidden and cell state
        '''
        
        input_embedd                           = self.embedding(input_sequence)
        lstm_state_h,lstm_state_c = states[0],states[1]
        self.lstm_output,lstm_state_h,lstm_state_c=self.lstm(input_embedd)
        return self.lstm_output,lstm_state_h,lstm_state_c

      

    
    def initialize_states(self,batch_size):
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      return [np.zeros((batch_size,self.lstm_size)),np.zeros((batch_size,self.lstm_size))]


In [123]:
class Decoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,out_vocab_size,embedding_size,lstm_size,input_length):

        #Initialize Embedding layer
        #Intialize Decoder LSTM layer
        super().__init__()
        self.out_vocab_size = out_vocab_size
        self.embedding_size = embedding_size
        self.lstm_size = lstm_size
        self.input_length = input_length
        # we are using embedding_matrix and not training the embedding layer
        self.embedding = tf.keras.layers.Embedding(input_dim=self.out_vocab_size, output_dim=self.embedding_size, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_decoder")
        self.gru = tf.keras.layers.GRU(self.lstm_size, return_sequences=True, return_state=True, name="Encoder_LSTM")
    
    def call(self,input_sequence,initial_states):
        '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to decoder_lstm
        
          returns -- decoder_output,decoder_final_state_h,decoder_final_state_c
        '''
        target_embedd = self.embedding(input_sequence)
        decoder_output,decoder_final_state_h = self.gru(target_embedd, initial_state=[initial_states[0]])
        return decoder_output,decoder_final_state_h

      


In [136]:
class Encoder_decoder(tf.keras.Model):
    
    def __init__(self,encoder_inputs_length,decoder_inputs_length, output_vocab_size,batch_size):
        
        #Create encoder object
        #Create decoder object
        #Intialize Dense layer(out_vocab_size) with activation='softmax'
        super().__init__() # https://stackoverflow.com/a/27134600/4084039
        self.batch_size=batch_size
        self.encoder = Encoder(vocab_size_source+1,512,256,encoder_inputs_length)
        self.decoder = Decoder(vocab_size_target+1,512,256,decoder_inputs_length)
        self.dense   = tf.keras.layers.Dense(output_vocab_size, activation='softmax')
    
    
    def call(self,data):
        '''
        A. Pass the input sequence to Encoder layer -- Return encoder_output,encoder_final_state_h,encoder_final_state_c
        B. Pass the target sequence to Decoder layer with intial states as encoder_final_state_h,encoder_final_state_C
        C. Pass the decoder_outputs into Dense layer 
        
        Return decoder_outputs
        '''
        input,output = data[0], data[1]
        initial_state=self.encoder.initialize_states(self.batch_size)
        encoder_output, encoder_h, encoder_c = self.encoder(input,initial_state)
        decoder_output,decoder_final_state_h= self.decoder(output,[encoder_h])
        output                               = self.dense(decoder_output)
        return output
        

In [137]:
class Dataset:
    def __init__(self, df, tknizer_source, tknizer_target, source_len,target_len):
        self.encoder_inps = df['source'].values
        self.decoder_inps = df['target_in'].values
        self.decoder_outs = df['target_out'].values
        self.tknizer_target = tknizer_target
        self.tknizer_source = tknizer_source
        self.source_len = source_len
        self.target_len = target_len


    def __getitem__(self, i):
        self.encoder_seq = self.tknizer_source.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.tknizer_target.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.tknizer_target.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.source_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.target_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.target_len, dtype='int32', padding='post')
        self.one_hot_encoded = np.zeros((len(self.decoder_out_seq),self.target_len,vocab_size_target),dtype='float32')

        for i,sentence in enumerate(self.decoder_out_seq):
          for j,word in enumerate(sentence):
            self.one_hot_encoded[i,j,word]=1
        



        return self.encoder_seq, self.decoder_inp_seq, self.one_hot_encoded

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)

    
class Dataloder(tf.keras.utils.Sequence):    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        return tuple([[batch[0],batch[1]],batch[2]])

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

In [138]:
train_dataset = Dataset(train, tknizer_source, tknizer_target,170,200)
test_dataset  = Dataset(validation, tknizer_source, tknizer_target,170,200)

train_dataloader = Dataloder(train_dataset, batch_size=512)
test_dataloader = Dataloder(test_dataset, batch_size=20)


print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)

(512, 170) (512, 200) (512, 200, 92)


In [139]:
def changeLearningRate(epoch,lr):
    if epoch % 3 == 0:
      return lr*(0.5)
    return lr

In [140]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,LearningRateScheduler
import datetime

early_stop = EarlyStopping(monitor='val_loss', patience=4, verbose=1)
check_point = ModelCheckpoint('best_model_1.h5', monitor='val_loss',  verbose=1, save_best_only=True, mode='min',save_format='tf')
lrschedule = LearningRateScheduler(changeLearningRate, verbose=1)

In [141]:
#Create an object of encoder_decoder Model class, 
# Compile the model and fit the model
model  = Encoder_decoder(encoder_inputs_length=170,decoder_inputs_length=200,output_vocab_size=vocab_size_target,batch_size=512)
optimizer = tf.keras.optimizers.Adam(0.01)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
train_steps=train.shape[0]//512
valid_steps=validation.shape[0]//20
model.fit_generator(train_dataloader, steps_per_epoch=train_steps, epochs=100, validation_data=test_dataloader, validation_steps=valid_steps,callbacks=[early_stop,lrschedule])
model.summary()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/100

Epoch 00001: LearningRateScheduler reducing learning rate to 0.004999999888241291.
3/3 [==============================] - 71s 35s/step - loss: 1.5354 - accuracy: 0.1540 - val_loss: 1.4060 - val_accuracy: 0.1758
Epoch 2/100

Epoch 00002: LearningRateScheduler reducing learning rate to 0.004999999888241291.
3/3 [==============================] - 2s 506ms/step - loss: 1.2939 - accuracy: 0.1434 - val_loss: 1.0846 - val_accuracy: 0.2136
Epoch 3/100

Epoch 00003: LearningRateScheduler reducing learning rate to 0.004999999888241291.
3/3 [==============================] - 2s 509ms/step - loss: 1.1312 - accuracy: 0.2131 - val_loss: 1.0058 - val_accuracy: 0.2218
Epoch 4/100

Epoch 00004: LearningRateScheduler reducing learning rate to 0.0024999999441206455.
3/3 [==============================] - 2s 512ms/step - loss: 1.0474 - accuracy: 0.2404 - val_loss: 0.9583 - val_accuracy: 0.2515
Epoch 5/100

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0024999999441206455.
3/3

In [143]:
batch_size=512
units=256

In [144]:
def predict(input_sentence):

  '''
  A. Given input sentence, convert the sentence into integers using tokenizer used earlier
  B. Pass the input_sequence to encoder. we get encoder_outputs, last time step hidden and cell state
  C. Initialize index of <start> as input to decoder. and encoder final states as input_states to decoder
  D. till we reach max_length of decoder or till the model predicted word <end>:
         predicted_out,state_h,state_c=model.layers[1](dec_input,states)
         pass the predicted_out to the dense layer
         update the states=[state_h,state_c]
         And get the index of the wordcc with maximum probability of the dense layer output, using the tokenizer(word index) get the word and then store it in a string.
         Update the input_to_decoder with current predictions
  F. Return the predicted sentence
  '''
  initial_state_enc=[np.zeros((batch_size,units)),np.zeros((batch_size,units))]
  inp_seq = tknizer_source.texts_to_sequences([input_sentence])
  inp_seq = pad_sequences(inp_seq,padding='post',maxlen=170)

  en_outputs,state_h , state_c = model.layers[0](tf.constant(inp_seq),initial_state_enc)
  cur_vec = tf.constant([[tknizer_target.word_index['\t']]])
  pred = []
  #Here 200 is the max_length of the sequence
  for i in range(200):
    infe_output, state_h = model.layers[1](cur_vec,[state_h])
    infe_output = model.layers[2](infe_output)
    cur_vec = np.reshape(np.argmax(infe_output), (1, 1))
    pred.append(tknizer_target.index_word[cur_vec[0][0]])
    if(pred[-1]=='\n'):
      break
    translated_sentence = ''.join(pred)

  return translated_sentence

In [147]:
for i in validation['source']:
  print("The Actual Output is:")
  print(i)
  print("The Predicted Output is:")
  pred=predict(i)
  print(pred)
  print('>'*100)


The Actual Output is:
Hey u called me huh.. Wat's up leh?
The Predicted Output is:
He ang tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou t
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Actual Output is:
u got cash card w u now.. xin wan to photostat something..
The Predicted Output is:
He ang tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou t
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
The Actual Output is:
I registered 4 it liao... I put class 13 first den class 14...
The Predicted Output is:
He ang tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou tou to

In [148]:
# Predict on 1000 random sentences on test data and calculate the average BLEU score of these sentences.
import nltk.translate.bleu_score as bleu
bleu_scores_lst=[]
for i in validation[:]['source']:
  reference = [i.split(),] # the original
  predicted=predict(i)
  translation = predicted.split()
  values=bleu.sentence_bleu(reference, translation)
  bleu_scores_lst.append(values)

# https://www.nltk.org/_modules/nltk/translate/bleu_score.html

In [149]:
average_bleu_scores=sum(bleu_scores_lst)/len(bleu_scores_lst)
print("Average BLEU score of these 1000 test data sentences is: ",average_bleu_scores)

Average BLEU score of these 1000 test data sentences is:  0.0


In [150]:
bleu_scores_lst

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [1]:
from prettytable import PrettyTable
table1=PrettyTable()

In [2]:
table1.field_names=['S.NO','MODEL','Average_Bleu_Score']
table1.add_row([1,'Simple_ManytoMany_Characterlevel_LSTM_Model',0.32245004727098137])
table1.add_row([2,'Simple_ManytoMany_Characterlevel_Bidirectional_LSTM_Model',0.289174940583241])
table1.add_row([3,'Simple_ManytoMany_Wordlevel_LSTM_Model',0.3605904404428826])
table1.add_row([4,'Simple_ManytoMany_Wordlevel_Bidirectional_LSTM_Model',0.39176783220696243])
table1.add_row([5,'Simple_ManytoMany_Wordlevel_LSTM_Model(Fasttext_Embeddings)',0.44321749874121313])
table1.add_row([6,'Simple_ManytoMany_Wordlevel_Bidirectional_LSTM_Model(Fasttext_Embeddings)',0.48304227467563887])
table1.add_row([7,'Encoder_Decoder_Wordlevel_LSTM',0.17936797861081147])
table1.add_row([8,'Encoder_Decoder_Characterlevel_LSTM',0.1318701360495755])
table1.add_row([9,'Encoder_Decoder_Attention_Wordlevel_LSTM',0.010138124061366445])
table1.add_row([10,'Encoder_Decoder_Attention_Characterlevel_LSTM',0.015891448522335924])
table1.add_row([11,'Encoder_Decoder_Attention_Wordlevel_LSTM(Fasttext_Embeddings)',0.03251957333271974])
table1.add_row([12,'Monotonic_Attention',0.19536971776642903])

In [3]:
print(table1)

+------+---------------------------------------------------------------------------+----------------------+
| S.NO |                                   MODEL                                   |  Average_Bleu_Score  |
+------+---------------------------------------------------------------------------+----------------------+
|  1   |                Simple_ManytoMany_Characterlevel_LSTM_Model                | 0.32245004727098137  |
|  2   |         Simple_ManytoMany_Characterlevel_Bidirectional_LSTM_Model         |  0.289174940583241   |
|  3   |                   Simple_ManytoMany_Wordlevel_LSTM_Model                  |  0.3605904404428826  |
|  4   |            Simple_ManytoMany_Wordlevel_Bidirectional_LSTM_Model           | 0.39176783220696243  |
|  5   |        Simple_ManytoMany_Wordlevel_LSTM_Model(Fasttext_Embeddings)        | 0.44321749874121313  |
|  6   | Simple_ManytoMany_Wordlevel_Bidirectional_LSTM_Model(Fasttext_Embeddings) | 0.48304227467563887  |
|  7   |                    